<a href="https://colab.research.google.com/github/yanglinjing/dand_p7_data_wragling/blob/master/wrangle_act2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Wrangle WeRateDogs Twitter Data

# Goal
Wrangle **WeRateDogs Twitter** data to create interesting and trustworthy analyses an visualizations. 

In [0]:
import pandas as pd
import requests
import csv
import tweepy
import os
import json

In [4]:
# Check current working directory

!pwd

/content


In [5]:
# All the documents are stored on Google Drive.
# It requires authorisation before use them.
# This chunk of code is about authorisation.

# Mounting Google Drive locally
# https://colab.research.google.com/notebooks/io.ipynb#scrollTo=u22w3BFiOveA

from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [6]:
# Change Working Directory in Google Drive

os.chdir('gdrive/My Drive/Colab Notebooks/dand_p7_data_wragling')
!pwd

# Entire Url: '/content/gdrive/My Drive/Colab Notebooks/dand_p7_data_wragling'

/content/gdrive/My Drive/Colab Notebooks/dand_p7_data_wragling


In [0]:
# Create a new folder if it does not exist

folder_name = 'raw_data'
if not os.path.exists(folder_name):
  os.makedirs(folder_name)

# Data Gathering
## Enhanced Twitter Archive
It contains basic tweet data for all 5000+ of their tweets, including rating, dog name, and dog "stage" (i.e. doggo, floofer, pupper, and puppo) .

 Of the 5000+ tweets, it is filtered for tweets with ratings only (there are 2356).
 
 The ratings,  dog names and dog stages probably aren't all correct.
 
The rating numerators are greater than the denominators. This unique rating system is a big part of the popularity of WeRateDogs.

In [0]:
### 1st df: WeRateDogs Twitter archive

df_1_path = os.path.join(folder_name, 'twitter-archive-enhanced.csv')

with open(df_1_path, 'r') as f:
  df_1 = pd.read_csv(f)

In [9]:
df_1.head(1)

,tweet_id,in_reply_to_status_id,in_reply_to_user_id,timestamp,source,text,retweeted_status_id,retweeted_status_user_id,retweeted_status_timestamp,expanded_urls,rating_numerator,rating_denominator,name,doggo,floofer,pupper,puppo
0,892420643555336193,NaN,NaN,2017-08-01 16:23:56 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Phineas. He's a mystical boy. Only eve...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/892420643...,13,10,Phineas,None,None,None,None


In [10]:
df_1.shape

(2356, 17)

## Additional Data via the Twitter API
**Retweet count** and **favorite count** are two of the notable column omissions of Twitter archives.

Fortunately, this additional data can be gathered from Twitter's API. 

In [0]:
### 2nd df: 
# Each tweet's retweet count and favorite ("like") count

# Using the tweet IDs in the WeRateDogs Twitter archive, 
# query the Twitter API for each tweet's JSON data 
# using Python's Tweepy library:
# http://www.tweepy.org/

consumer_key = 'pTzzUxLTJcdNCeAHOvhUbflHX'
consumer_secret = 'bW1RvcK3vlFBxYFe9XozytZTR50hVsZmdpRCBV2zKFKUSBYEOG'
access_token = '1090182192024629249-EMhWG3aaCj1QPeMd8bHO21XPYDRQse'
access_secret = '0qWGdlA4iCQ7wFw0RK1FXC6XKHfXtzYryvil0wcyzmN3j'

auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_secret)

api = tweepy.API(auth)

# Note: do not include your Twitter API keys, secrets, and tokens

In [0]:
# Use id provided in df_1
# to retrive its retweet & favorite count, and
# temporarily store in a list

df_2_list = []

# Use each id in df_1
for id in df_1['tweet_id']:
  
  try:
    # get status object
    status = api.get_status(id) 

    # get json strings from status obj
    json_str = json.loads(json.dumps(status._json))

    # get retweet/like counts from json strings
    retweet_count = json_str['retweet_count']
    favorite_count = json_str['favorite_count']
    
    # write to list
    df_2_list.append(f"{id} {retweet_n} {favourate_n}")

  # in case there is no status obj returned
  except Exception as e:
    df_2_list.append(f"{id} Nan Nan")


In [0]:
# store each tweet's info in a file called tweet_json.txt file. 
# Each tweet's JSON data should be written to its own line. 

df_2_path = os.path.join(folder_name, 'tweet_json.txt')

with open(df_2_path, "w") as f:
  for item in df_2_list:
    f.write(item + '\n')
    

In [0]:
# Read this .txt file line by line into a pandas DataFrame 
# with tweet ID, retweet count, and favorite count. 

df_2 = pd.read_csv(df_2_path,  sep=" ", 
                  names = ['id', 'retweet_n', 'favourate_n'])


In [21]:
df_2.head()

,id,retweet_n,favourate_n
0,892420643555336193,Nan,Nan
1,892177421306343426,Nan,Nan
2,891815181378084864,Nan,Nan
3,891689557279858688,Nan,Nan
4,891327558926688256,Nan,Nan


In [22]:
df_2.shape

(2356, 3)

## Image Predictions File

Every image in the WeRateDogs Twitter archive was through a neural network that can classify breeds of dogs.

(The neural network: https://www.youtube.com/watch?v=2-Ol7ZB0MmU)

The results: a table full of image predictions (the top three only) alongside each tweet ID, image URL, and the image number that corresponded to the **most confident prediction** (**numbered 1 to 4** since tweets can have up to four images).

In the table,
- **tweet_id** is the last part of the tweet URL after "status/" → https://twitter.com/dog_rates/status/889531135344209921
- **p1** is the algorithm's **#1 prediction** for the image in the tweet → golden retriever
- **p1_conf** is how confident the algorithm is in its #1 prediction → 95%
- **p1_dog** is whether or not the #1 prediction is a breed of dog → TRUE
- **p2** is the algorithm's **2nd most likely prediction** → Labrador retriever
- -etc.

In [23]:
### 3rd df: tweet image predictions

# The tweet image predictions, i.e., what breed of dog (or other object, animal, etc.) 
# is present in each tweet according to a neural network. 

# This file (image_predictions.tsv) is hosted on Udacity's servers and 
# should be downloaded programmatically using the Requests library 
# Requests library: http://docs.python-requests.org/en/master/


# Store the content into response in working memory

url_3 = 'https://d17h27t6h515a5.cloudfront.net/topher/2017/August/599fd2ad_image-predictions/image-predictions.tsv'
response = requests.get(url_3)

response
# <Response [200]>: success

<Response [200]>

In [0]:
# folder name / file name (shown in url_2):
# 'raw_data/image-predictions.tsv'

df_3_path = os.path.join(folder_name, url_3.split('/')[-1])

# put the content of response into f
with open (df_3_path, mode='wb') as f:
  f.write(response.content)
  
# ensure the new file has been built
# os.listdir(folder_name)

In [25]:
df_3 = pd.read_csv(df_3_path, sep='\t')

df_3.head()

,tweet_id,jpg_url,img_num,p1,p1_conf,p1_dog,p2,p2_conf,p2_dog,p3,p3_conf,p3_dog
0,666020888022790149,https://pbs.twimg.com/media/CT4udn0WwAA0aMy.jpg,1,Welsh_springer_spaniel,0.465074,True,collie,0.156665,True,Shetland_sheepdog,0.061428,True
1,666029285002620928,https://pbs.twimg.com/media/CT42GRgUYAA5iDo.jpg,1,redbone,0.506826,True,miniature_pinscher,0.074192,True,Rhodesian_ridgeback,0.072010,True
2,666033412701032449,https://pbs.twimg.com/media/CT4521TWwAEvMyu.jpg,1,German_shepherd,0.596461,True,malinois,0.138584,True,bloodhound,0.116197,True
3,666044226329800704,https://pbs.twimg.com/media/CT5Dr8HUEAA-lEu.jpg,1,Rhodesian_ridgeback,0.408143,True,redbone,0.360687,True,miniature_pinscher,0.222752,True
4,666049248165822465,https://pbs.twimg.com/media/CT5IQmsXIAAKY4A.jpg,1,miniature_pinscher,0.560311,True,Rottweiler,0.243682,True,Doberman,0.154629,True


In [26]:
df_3.shape

(2075, 12)

## 8  quality issues 

## 2 tidiness issues 

do not need to gather the tweets beyond August 1st, 2017